In [ ]:
%config Completer.use_jedi = False # To make auto-complete faster

#Reloads imported files automatically
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../src/')

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:87% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
import time
import os

import pynbody

In [ ]:
import matplotlib.pyplot as plt

from matplotlib_param_funcs import set_matplotlib_params,reset_rcParams
set_matplotlib_params()

In [ ]:
import miscellaneous_functions as MF
import mixed_plots as MP
import from_sim_to_numpy as sim_to_np

In [ ]:
general_path = '/Users/luismi/Desktop/MRes_UCLan/'

# 708main

In [ ]:
filename = "708main"

simulation_path = general_path+f"data/{filename}/run{filename}.01000.gz"

print(simulation_path)
assert os.path.isfile(simulation_path), "Directory does not exist"

In [ ]:
save_path = general_path + f"data/{filename}/numpy_arrays/"
MF.create_dir(save_path)

print(save_path)

In [ ]:
R0=8.1
pos_factor = 1.7
zabs = True
GSR = True

I_radius = 4 if filename == "708main" else 2

axisymmetric = False

In [ ]:
#min_angle = 0
#max_angle = 90
#step = 90
#angle_list = np.arange(min_angle, max_angle + step, step)
#angle_list = np.append(angle_list, 27)

angle_list = [27] #[0,15,20,25,27,30,35,40,45,90]
print(angle_list)

In [ ]:
sim_to_np.load_process_and_save(simulation_path, save_path, angle_list=angle_list, pos_factor=pos_factor, zabs=zabs, GSR=GSR,R0=R0,\
                                axisymmetric=axisymmetric,I_radius=I_radius,fileprefix=filename)

## Step by step

### Load

In [ ]:
sim = sim_to_np.load_pynbody_sim(simulation_path)

aligned = False
rotated = False

In [ ]:
def extract_xyz_df_from_sim_stars(stars):
    df = pd.DataFrame()
    df["x"],df["y"],df["z"] = np.moveaxis(a=stars['pos'].in_units('kpc'), source=1, destination=0)

    return df

In [ ]:
I_radius = 2

bar_angle = sim_to_np.compute_bar_angle(sim.s,I_radius=I_radius)

print("Bar angle:",bar_angle)

In [ ]:
fig,(ax,az)=plt.subplots(figsize=(8,8),nrows=2)

xymax = 6

MP.quick_show_xy(extract_xyz_df_from_sim_stars(sim.s),ax=ax,xmin=-xymax,xmax=xymax,ymin=-xymax,ymax=xymax)
MP.quick_show_xz(extract_xyz_df_from_sim_stars(sim.s),ax=az,xmin=-xymax,xmax=xymax,zmin=0,zmax=3)
plt.show()

In [ ]:
fig,(ax,az)=plt.subplots(figsize=(8,8),nrows=2)

bar_stars = sim_to_np.get_bar_stars(sim)

MP.quick_show_xy(extract_xyz_df_from_sim_stars(bar_stars),ax=ax)
MP.quick_show_xz(extract_xyz_df_from_sim_stars(bar_stars),ax=az)
plt.show()

### Non-axisymmetric only

#### Align

In [ ]:
# Rotating in-place because I could not find a way to deep-copy the simulation - see https://stackoverflow.com/questions/58415397/

if aligned:
    print("Already aligned")
else:    
    sim.rotate_z(-bar_angle)
    print("Aligned successfully")

In [ ]:
bar_angle_after_alignment = sim_to_np.compute_bar_angle(sim.s,I_radius=I_radius)

print(bar_angle_after_alignment)

In [ ]:
xymax = 5

fig,axs=plt.subplots(figsize=(8,8),nrows=2)
MP.quick_show_xy(extract_xyz_df_from_sim_stars(sim.s),ax=axs[0],xmin=-xymax,xmax=xymax,ymin=-xymax,ymax=xymax)
MP.quick_show_xz(extract_xyz_df_from_sim_stars(sim.s),ax=axs[1],xmin=-xymax,xmax=xymax,zmin=0,zmax=3)
plt.show()

#### Rotate by bar angle

In [ ]:
rot_angle = 27

if rotated:
    print("Already rotated")
else:
    sim.rotate_z(rot_angle)
    print("Rotated successfully")

In [ ]:
bar_angle_after_rotation = sim_to_np.compute_bar_angle(sim,I_radius=I_radius)

print(bar_angle_after_rotation)

In [ ]:
xymax = 5

fig,axs=plt.subplots(figsize=(8,8),nrows=2)
MP.quick_show_xy(extract_xyz_df_from_sim_stars(sim.s),ax=axs[0],xmin=-xymax,xmax=xymax,ymin=-xymax,ymax=xymax)
MP.quick_show_xz(extract_xyz_df_from_sim_stars(sim.s),ax=axs[1],xmin=-xymax,xmax=xymax,zmin=0,zmax=3)
plt.show()

### Convert to df

In [ ]:
# axisymmetric = True
axisymmetric = False

if axisymmetric:
    bar_angle = 27 # whatever

In [ ]:
df = sim_to_np.convert_sim_to_df(sim, angle=bar_angle, axisymmetric=axisymmetric)

In [ ]:
df.head()

In [ ]:
MP.quick_show_xy_xz(df)

In [ ]:
MP.visually_inspect_bar_angle(df, zmin=0.5,age_lims=[4,7],bar_angle=bar_angle if not axisymmetric else None)

## Load numpy
Just to test the saved file came out as expected

In [ ]:
import load_sim

In [ ]:
dummy_df = pd.DataFrame([[1,2],[4,3]],columns=['a','b'])

In [ ]:
print(general_path)

In [ ]:
R0 = 8.1

np_path = general_path+f"data/708main_simulation/numpy_arrays/R0_{R0}/"

**Non-axisymmetric**

In [ ]:
df0 = load_sim.load_simulation(np_path,GSR=GSR,R0=R0,zabs=zabs,pos_factor=pos_factor,axisymmetric=False)

In [ ]:
MP.quick_show_xy_xz(df=df0)

In [ ]:
MP.visually_inspect_bar_angle(df0, xymax=5, zmin=0.5, age_lims=[4,7],bins=50, bar_angle=27)

**Axisymmetrise here**

In [ ]:
sim_to_np.axisymmetrise(df0)


In [ ]:
MP.quick_show_xy_xz(df0)

In [ ]:
MP.visually_inspect_bar_angle(df0, xymax=5,zmin=0.5,age_lims=[4,7],bins=50)

**Load axisymmetric**

In [ ]:
axi_df = load_sim.load_simulation(np_path,GSR=GSR,R0=R0,zabs=zabs,pos_factor=pos_factor,axisymmetric=True)

In [ ]:
MP.quick_show_xy_xz(axi_df)

In [ ]:
MP.visually_inspect_bar_angle(axi_df, xymax=5,zmin=0.5,age_lims=[4,7],bins=50)

# Joao

In [ ]:
joao_path = '/home/luismi98/MRes_UCLan/dwarfM06Y193/'
accreted_filepath = '/home/luismi98/MRes_UCLan/dwarfM06Y193Zn51Vx30Rotn15x_pid_accreted.npy'

save_path = '/home/luismi98/MRes_UCLan/joao_numpy_arrays/'

In [ ]:
filename = 'jrun002.dwarfM06Y193Zn51Vx30Rotn15x.01000.gz'

load_path = joao_path+filename

In [ ]:
# Load and align
joao_sim = pynbody.load(load_path)
joao_sim.physical_units()
pynbody.analysis.angmom.faceon(joao_sim.stars)

pynbody.plot.stars.render(joao_sim.s, width='15 kpc')
plt.savefig(save_path+filename+'.png',dpi=300)

In [ ]:
indexes = np.arange(0,len(joao_sim.s),1)
joao_sim.s['pid'] = indexes

pid_acc = np.load(accreted_filepath)

accreted = np.isin(joao_sim.s['pid'], pid_acc)
joao_sim.s['accreted'] = accreted

print(np.sum(accreted)*100/len(joao_sim.s['pid']),"% are accreted")

In [ ]:
joao_df = pd.DataFrame()

for key in joao_sim.s.keys():
    if key in ['vel','pos','eps']: continue
    joao_df[key] = np.array(joao_sim.s[key])
    
joao_df['age'] = joao_df['tform'].max() - joao_df['tform']

In [ ]:
print("The dataframe occupies",joao_df.memory_usage(index=True).sum()/10**6,"Mb")

print("The dataframe in np.float32 occupies",joao_df.astype(np.float32).memory_usage(index=True).sum()/10**6,"Mb")

In [ ]:
columns = list(joao_df.columns)

In [ ]:
#savetype = np.float64
savetype = np.float32

In [ ]:
save_name = 'ALL_jrun002.dwarfM06Y193Zn51Vx30Rotn15x.01000.npy'

np.save(save_path + save_name, np.array(joao_df.values.astype(savetype)))

print("Saved as type",savetype)

In [ ]:
with open(save_path + 'columns.txt', 'w') as f:
    string = f"Saved simulation with datatype {savetype}.\nThe columns are:{columns}"
    f.write(string)